In [1]:
import numpy as np
import tensorflow as tf


from tensorflow import keras
from tensorflow.keras import layers

## Create Simulated Data

In [2]:
def create_data(n=100):
    """
        f(x1, x2) = 5 * x_1**2 + 4 * x_2**2 - 3 * x_1 * x_2
    """
    ret = np.zeros(shape=(n, 3)).astype("float32")
    x = np.random.rand(n, 2)
    y = (5 * x[:, 0]**2 + 4 * x[:, 1]**2 - 3 * x[:, 0] * x[:, 1]).reshape(-1, 1)
    ret[:, [0, 1]] = x
    ret[:, [2]] = y
    
    return ret

## Test Out Function

In [3]:
d = create_data(4)

In [4]:
d

array([[0.8351567 , 0.07995834, 3.3126736 ],
       [0.08991358, 0.5718037 , 1.1940213 ],
       [0.01917329, 0.47058797, 0.860582  ],
       [0.0115176 , 0.696974  , 1.9196717 ]], dtype=float32)

In [5]:
val = round(5 * d[0, 0]**2 + 4 * d[0, 1]**2 - 3 * d[0, 0] * d[0, 1], 4)
exp_val = round(d[0, 2], 4)

val, exp_val

(3.3127, 3.3127)

## Create Network

The network that we will create will have two dense layers with ReLU activations. The goal is to see how many of these neurons become "dead".

### Train and Test Data

In [6]:
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import Normalization

NUM_SAMPLES = 1000
TRAIN_FRAC = 0.60
VAL_FRAC = 0.20

data = create_data(n=1000)

train_idx = int(TRAIN_FRAC * NUM_SAMPLES)
val_idx = train_idx + int(VAL_FRAC * NUM_SAMPLES)

train_data = data[: train_idx]
val_data = data[train_idx: val_idx]
test_data = data[val_idx: ]

### Normalizer 

In [7]:
def normalize(data: np.array, mean: np.array=None, sd: np.array=None):
    if (mean is not None) and (sd is not None):
        assert data.shape[1] == len(mean) == len(sd)
    
    if mean is None:
        mean = data.mean(axis=0)
    if sd is None:
        sd = data.std(axis=0)
    
    normed = (data - mean) / sd
    
    return mean, sd, normed

In [8]:
mean, sd, norm_train_data = normalize(train_data)
_, _, norm_val_data = normalize(val_data, mean=mean, sd=sd)
_, _, norm_test_data = normalize(test_data, mean=mean, sd=sd)

### Model Layers

In [9]:
inputs = keras.Input(shape=2)
dense_1 = layers.Dense(units=10, activation='relu')(inputs)
dense_2 = layers.Dense(units=10, activation='relu')(dense_1)
output = layers.Dense(units=1, activation=None)(dense_2)

In [10]:
model = keras.Model(inputs=inputs, outputs=output)
model.compile(optimizer='rmsprop', loss='mean_squared_error')

In [11]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 2)]               0         
_________________________________________________________________
dense (Dense)                (None, 10)                30        
_________________________________________________________________
dense_1 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 151
Trainable params: 151
Non-trainable params: 0
_________________________________________________________________


In [12]:
batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices((test_data[:, [0, 1]], test_data[:, [2]])).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((val_data[:, [0, 1]], val_data[:, [2]])).batch(batch_size)

history = model.fit(train_dataset,
                    batch_size=32, 
                    epochs=10, validation_data=val_dataset)

Epoch 1/10
7/7 [==============================] - 1s 45ms/step - loss: 5.2889 - val_loss: 5.8107
Epoch 2/10
7/7 [==============================] - 0s 5ms/step - loss: 4.9126 - val_loss: 5.5038
Epoch 3/10
7/7 [==============================] - 0s 5ms/step - loss: 4.6443 - val_loss: 5.2259
Epoch 4/10
7/7 [==============================] - 0s 4ms/step - loss: 4.3963 - val_loss: 4.9483
Epoch 5/10
7/7 [==============================] - 0s 4ms/step - loss: 4.1489 - val_loss: 4.6721
Epoch 6/10
7/7 [==============================] - 0s 4ms/step - loss: 3.9061 - val_loss: 4.3971
Epoch 7/10
7/7 [==============================] - 0s 5ms/step - loss: 3.6668 - val_loss: 4.1230
Epoch 8/10
7/7 [==============================] - 0s 5ms/step - loss: 3.4273 - val_loss: 3.8495
Epoch 9/10
7/7 [==============================] - 0s 4ms/step - loss: 3.1899 - val_loss: 3.5787
Epoch 10/10
7/7 [==============================] - 0s 4ms/step - loss: 2.9573 - val_loss: 3.3145


In [13]:
pred = model.predict(test_data[:, [0, 1]])

In [14]:
np.mean((pred - test_data[:, [2]])**2)

2.9381568

## Weights

In [15]:
for layer in model.layers: 
    print(layer.get_config(), layer.get_weights())

{'batch_input_shape': (None, 2), 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'input_1'} []
{'name': 'dense', 'trainable': True, 'dtype': 'float32', 'units': 10, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None} [array([[ 0.58302754, -0.38940164, -0.4515814 , -0.67748547,  0.12296874,
        -0.22077397, -0.3952442 ,  0.39272025, -0.00256889,  0.19800133],
       [ 0.02126208, -0.27769834, -0.48752052, -0.25400287, -0.6434514 ,
        -0.20385504,  0.5445364 ,  0.23974535,  0.2696259 , -0.5468051 ]],
      dtype=float32), array([ 0.08030444,  0.        ,  0.        ,  0.        , -0.04867382,
        0.        ,  0.08215104,  0.07784649, -0.06343292, -0.05424789],
      dtype=float32)]
{'name': 'dense_1', 